In [ ]:
# necessary for DenodoConnection
%pip install psycopg2-binary

In [3]:
# import packages
import os
import sys
import pandas as pd
import numpy as np

# need to add dme_sagemaker to path to load in libraries
sys.path.append("/root/dme_sagemaker/dme_sagemaker")

from libs.denodo.denodo_connection import DenodoConnection
import libs.performance_lib.performance_sql_recipes as performance_sql_recipes
from libs.helper.parameters_helper import ParametersHelper
from libs.performance_lib import performance_helper

ENVIRONMENT name from config_vars: uat
DME_PROJECT name from config_vars: None


In [4]:
ph = ParametersHelper('denodo')
print(ph.get_parameter('username'),
      ph.get_parameter('server'))

h1101871@syngenta.org uat.denodo.syngentadigitalapps.com


In [6]:
query_str = """
    select * from
    managed.rv_trial_pheno_analytic_dataset
    limit 10
"""

with DenodoConnection() as dc:
    output_df = dc.get_data(query_str)

/opt/conda/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
output_df.head()

,crop,season,geo_segment,country_name,country_code,ap_data_sector,year,bg_name,bg_code,crop_type_code,...,cregf,crtnf,psp_exclude,entry_no,trait_measure_code,result_numeric_value,result_alpha_value,result_date_value,pr_exclude,pr_last_chg_date
0,Sunflower,SUMR,None,France,FR,SUNFLOWER_EAME_SUMMER,2020.0,French ( Trial ),FR,YG,...,0.0,0.0,False,1.0,NOTET,NaN,HT/VG/REG,NaT,False,2020-09-18 12:29:48.330
1,Sunflower,SUMR,None,France,FR,SUNFLOWER_EAME_SUMMER,2020.0,French ( Trial ),FR,YG,...,0.0,0.0,False,7.0,FERRN,1.0,None,NaT,False,2020-07-21 12:05:58.940
2,Sunflower,SUMR,None,France,FR,SUNFLOWER_EAME_SUMMER,2020.0,French ( Trial ),FR,YG,...,0.0,0.0,False,19.0,PLTAR,6.0,None,NaT,False,2020-09-18 12:29:48.330
3,Sunflower,SUMR,None,France,FR,SUNFLOWER_EAME_SUMMER,2020.0,French ( Trial ),FR,YG,...,0.0,0.0,False,2.0,HARVD,NaN,None,2020-09-08,False,2020-09-16 18:22:59.023
4,Sunflower,SUMR,None,France,FR,SUNFLOWER_EAME_SUMMER,2020.0,French ( Trial ),FR,YG,...,0.0,0.0,False,2.0,PLHTN,170.0,None,NaT,False,2020-07-24 09:26:14.537


In [30]:
def merge_pvs_input(ap_data_sector, analysis_year, analysis_type):
    query_str1 = """
    -- select the single year PVS data with the appropriate source_id and apply config tables
    WITH "pvs_single_year" AS(
        SELECT 
            "rv_ap_all_pvs"."ap_data_sector",
            "rv_ap_all_pvs"."analysis_type",
            "rv_ap_all_pvs"."source_id",
            CASE 
                WHEN "rv_ap_all_pvs"."ap_data_sector" = 'SOY_NA_SUMMER' AND "rv_ap_all_pvs"."analysis_type" = 'MultiYear'
                    THEN 'all'
                WHEN "rv_ap_all_pvs"."market_seg" IS NULL OR "rv_ap_all_pvs"."market_seg" = ''
                    THEN 'all'
                ELSE LOWER("rv_ap_all_pvs"."market_seg")
            END AS "market_seg",
            "rv_ap_all_pvs"."trait",
            "rv_ap_all_pvs"."model",
            "rv_ap_all_pvs"."entry_identifier",
            LOWER("rv_ap_all_pvs"."material_type") AS "material_type",
            "rv_ap_all_pvs"."count",
            "rv_ap_all_pvs"."prediction",
            "rv_ap_all_pvs"."stderr",
            "dg_rm_list"."decision_group_rm",
            "dg_rm_list"."stage",
            CASE 
                WHEN "rv_ap_all_pvs"."ap_data_sector" = 'SOY_NA_SUMMER' AND "rv_ap_all_pvs"."analysis_type" = 'MultiYear'
                    THEN LOWER("rv_ap_all_pvs"."market_seg")
                WHEN "dg_rm_list"."technology" IS NULL OR "dg_rm_list"."technology" = ''
                    THEN 'all'
                ELSE LOWER("dg_rm_list"."technology")
            END AS "technology",
            "rv_ap_sector_trait_config"."conv_operator",
            "rv_ap_sector_trait_config"."conv_factor",
            "rv_ap_sector_trait_config"."dme_metric",
            "rv_ap_sector_trait_config"."dme_chkfl",
            "rv_ap_sector_trait_config"."dme_reg_x",
            "rv_ap_sector_trait_config"."dme_reg_y",
            "rv_ap_sector_trait_config"."dme_rm_est",
            "rv_ap_sector_trait_config"."dme_weighted_trait"
          FROM (
            SELECT
                "ap_data_sector",
                "trialing_year" as "source_year",
                "analysis_type",
                "source_id",
                FIRST("pipeline_runid") AS "pipeline_runid",
                LOWER("market_seg") AS "market_seg",
                "trait",
                "model",
                "entry_identifier",
                "material_type",
                FIRST("count") AS "count",
                FIRST("prediction") AS "prediction",
                FIRST("stderr") AS "stderr"
              FROM "advancement"."rv_ap_all_pvs" "rv_ap_all_pvs"
            WHERE "ap_data_sector" = {0}
              AND CAST("trialing_year" AS integer) = {1}
              AND "rv_ap_all_pvs"."analysis_type" = {2}
              AND "rv_ap_all_pvs"."analysis_type" IN ('SingleExp', 'GenoPred', 'MultiExp', 'MynoET')
              AND "rv_ap_all_pvs"."loc" = 'ALL'
            GROUP BY
                "ap_data_sector",
                "trialing_year",
                "analysis_type",
                "source_id",
                LOWER("market_seg"),
                "trait",
                "model",
                "entry_identifier",
                "material_type"
            ORDER BY "pipeline_runid" DESC
        ) "rv_ap_all_pvs"
        INNER JOIN (
            SELECT
                "ap_data_sector_name",
                "analysis_year",
                "decision_group",
                AVG(CASE 
                    WHEN "decision_group_rm" IS NULL OR "decision_group_rm" = ''
                        THEN 0
                    ELSE "decision_group_rm"
                END) AS "decision_group_rm",
                MAX("stage") AS "stage",
                FIRST("technology") AS "technology"
              FROM "advancement"."rv_ap_sector_experiment_config"
            WHERE "ap_data_sector_name" = {0}
            GROUP BY
                "ap_data_sector_name",
                "analysis_year",
                "decision_group"
        ) "dg_rm_list"
          ON "rv_ap_all_pvs"."source_id" = "dg_rm_list"."decision_group"
        INNER JOIN "advancement"."rv_ap_sector_trait_config" "rv_ap_sector_trait_config"
          ON "rv_ap_sector_trait_config"."ap_data_sector_name" = "dg_rm_list"."ap_data_sector_name"
            AND "rv_ap_sector_trait_config"."analysis_year" = "dg_rm_list"."analysis_year"
            AND "rv_ap_sector_trait_config"."trait" = "rv_ap_all_pvs"."trait"
    )

    -- find the maturity group associated with the source ID from experiment_config and the entry_id's associated with the source_id, then grab the appropriate multiyear rows
    SELECT
        "pvs_single_year"."ap_data_sector",
        {2} AS "analysis_type",
        {1} AS "analysis_year",
        "pvs_single_year"."source_id",
        "pvs_single_year"."stage",
        "pvs_single_year"."decision_group_rm",
        LOWER("pvs_single_year"."market_seg") AS "pvs_market_seg",
        'all' AS "pvs_loc",
        'all' AS "pvs_my_source_id",
        "pvs_single_year"."technology" AS "exp_technology",
        "pvs_single_year"."trait",
        "pvs_single_year"."entry_identifier",
        "pvs_single_year"."material_type",
        "pvs_single_year"."count",
        CASE 
            WHEN "pvs_single_year"."conv_operator" = '*' AND "pvs_single_year"."material_type" LIKE '%entry' AND "pvs_single_year"."conv_factor" != 0 AND "pvs_single_year"."prediction" IS NOT NULL
                THEN "pvs_single_year"."prediction"/"pvs_single_year"."conv_factor"
            WHEN "pvs_single_year"."conv_operator" = '/' AND "pvs_single_year"."material_type" LIKE '%entry' AND "pvs_single_year"."conv_factor" != 0 AND "pvs_single_year"."prediction" IS NOT NULL
                THEN "pvs_single_year"."prediction"*"pvs_single_year"."conv_factor"
            ELSE "pvs_single_year"."prediction"
        END AS "prediction",
        CASE 
            WHEN "pvs_single_year"."conv_operator" = '*' AND "pvs_single_year"."material_type" LIKE '%entry' AND "pvs_single_year"."conv_factor" != 0 AND "pvs_single_year"."stderr" IS NOT NULL
                THEN "pvs_single_year"."stderr"/"pvs_single_year"."conv_factor"
            WHEN "pvs_single_year"."conv_operator" = '/' AND "pvs_single_year"."material_type" LIKE '%entry' AND "pvs_single_year"."conv_factor" != 0 AND "pvs_single_year"."stderr" IS NOT NULL
                THEN "pvs_single_year"."stderr"*"pvs_single_year"."conv_factor"
            ELSE "pvs_single_year"."stderr"
        END AS "stderr",
        "pvs_single_year"."dme_metric",
        "pvs_single_year"."dme_chkfl",
        "pvs_single_year"."dme_reg_x",
        "pvs_single_year"."dme_reg_y",
        "pvs_single_year"."dme_rm_est",
        "pvs_single_year"."dme_weighted_trait"
      FROM "pvs_single_year"
    """.format("'" + ap_data_sector + "'",
               analysis_year,
               "'" + analysis_type + "'"
               )

    if analysis_type != 'GenoPred':
        query_str2 = """
    UNION ALL
    SELECT
        "pvs_single_year"."ap_data_sector",
        {2} AS "analysis_type",
        {1} AS "analysis_year",
        "pvs_single_year"."source_id",
        "pvs_single_year"."stage",
        "pvs_single_year"."decision_group_rm",
        LOWER("rv_ap_all_pvs"."market_seg") AS "pvs_market_seg",
        CASE WHEN "rv_ap_all_pvs"."analysis_type" = 'MultiYear'
            THEN "rv_ap_all_pvs"."loc"
            ELSE LOWER("rv_ap_all_pvs"."loc")
        END AS "pvs_loc",
        "rv_ap_all_pvs"."source_id" AS "pvs_my_source_id",
        "pvs_single_year"."technology" AS "exp_technology",
        "rv_ap_all_pvs"."trait",
        "rv_ap_all_pvs"."entry_identifier",
        "pvs_single_year"."material_type",
        "rv_ap_all_pvs"."count",
        CASE 
            WHEN "pvs_single_year"."conv_operator" = '*' AND "rv_ap_all_pvs"."analysis_type" != 'GenoPred' AND "pvs_single_year"."conv_factor" != 0 AND "rv_ap_all_pvs"."prediction" IS NOT NULL
                THEN "rv_ap_all_pvs"."prediction"/"pvs_single_year"."conv_factor"
            WHEN "pvs_single_year"."conv_operator" = '/' AND "rv_ap_all_pvs"."analysis_type" != 'GenoPred' AND "pvs_single_year"."conv_factor" != 0 AND "rv_ap_all_pvs"."prediction" IS NOT NULL
                THEN "rv_ap_all_pvs"."prediction"*"pvs_single_year"."conv_factor"
            ELSE "rv_ap_all_pvs"."prediction"
        END AS "prediction",
        CASE 
            WHEN "pvs_single_year"."conv_operator" = '*' AND "rv_ap_all_pvs"."analysis_type" != 'GenoPred' AND "pvs_single_year"."conv_factor" != 0 AND "rv_ap_all_pvs"."stderr" IS NOT NULL
                THEN "rv_ap_all_pvs"."stderr"/"pvs_single_year"."conv_factor"
            WHEN "pvs_single_year"."conv_operator" = '/' AND "rv_ap_all_pvs"."analysis_type" != 'GenoPred' AND "pvs_single_year"."conv_factor" != 0 AND "rv_ap_all_pvs"."stderr" IS NOT NULL
                THEN "rv_ap_all_pvs"."stderr"*"pvs_single_year"."conv_factor"
            ELSE "rv_ap_all_pvs"."stderr"
        END AS "stderr",
        "pvs_single_year"."dme_metric",
        "pvs_single_year"."dme_chkfl",
        "pvs_single_year"."dme_reg_x",
        "pvs_single_year"."dme_reg_y",
        "pvs_single_year"."dme_rm_est",
        "pvs_single_year"."dme_weighted_trait"
    FROM (
        SELECT DISTINCT
            "ap_data_sector",
            "source_id",
            "entry_identifier",
            "material_type",
            "stage",
            "technology",
            "decision_group_rm",
            "trait",
            "conv_operator",
            "conv_factor",
            "dme_metric",
            "dme_chkfl",
            "dme_reg_x",
            "dme_reg_y",
            "dme_rm_est",
            "dme_weighted_trait"
          FROM "pvs_single_year"
    )"pvs_single_year"
    INNER JOIN "advancement"."rv_ap_all_pvs" "rv_ap_all_pvs"
      ON "pvs_single_year"."entry_identifier" = "rv_ap_all_pvs"."entry_identifier"
        AND "pvs_single_year"."trait" = "rv_ap_all_pvs"."trait"
        AND ((LOWER("rv_ap_all_pvs"."analysis_type") = 'multiyear') AND (LOWER("rv_ap_all_pvs"."loc") != 'all')) 
        AND "rv_ap_all_pvs"."ap_data_sector" = {0}
        AND CAST("rv_ap_all_pvs"."source_year" AS integer) = {1}
    """.format("'" + ap_data_sector + "'",
               analysis_year,
               "'" + analysis_type + "'"
               )
    else:
        query_str2 = ""

    with DenodoConnection() as dc:
        output_df = dc.get_data(query_str1+query_str2)

    # cast all hybrid material types to entry for consistency
    output_df['material_type'][output_df['material_type'] == 'hybrid'] = 'entry'
    # rename market seg columns
    output_df = output_df.rename(columns={
        'pvs_market_seg':'market_seg',
        'exp_technology':'technology'
    })
    return output_df

In [38]:
ap_data_sector = 'CORN_LAS_SUMMER'
analysis_year = '2022'

df = merge_pvs_input(ap_data_sector, analysis_year, 'GenoPred')
df.shape

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(61836, 22)

In [39]:
df.head()

,ap_data_sector,analysis_type,analysis_year,source_id,stage,decision_group_rm,market_seg,pvs_loc,pvs_my_source_id,technology,...,material_type,count,prediction,stderr,dme_metric,dme_chkfl,dme_reg_x,dme_reg_y,dme_rm_est,dme_weighted_trait
0,CORN_LAS_SUMMER,GenoPred,2022,22SUPMYG58A,5.0,8.0,las_corn_south,all,all,all,...,pool2,None,-15.870535,NaN,performance,cperf,False,True,0,0
1,CORN_LAS_SUMMER,GenoPred,2022,22SUPMYG58A,5.0,8.0,las_corn_south,all,all,all,...,entry,None,124.092749,0.008132,performance,cperf,False,True,0,0
2,CORN_LAS_SUMMER,GenoPred,2022,22SUPMYG58A,5.0,8.0,las_corn_south,all,all,all,...,entry,None,126.152319,0.007518,performance,cperf,False,True,0,0
3,CORN_LAS_SUMMER,GenoPred,2022,22SUPMYG58A,5.0,8.0,las_corn_south,all,all,all,...,entry,None,127.990563,0.007785,performance,cperf,False,True,0,0
4,CORN_LAS_SUMMER,GenoPred,2022,22SUPMYG58A,5.0,8.0,las_corn_south,all,all,all,...,entry,None,128.165408,0.007986,performance,cperf,False,True,0,0


In [41]:
df = performance_sql_recipes.get_material_by_trialstage_year_one_sector(ap_data_sector='SOY_LAS_SUMMER',min_year=2020,max_year=2022)

starting denodo connection
getting data


/opt/conda/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


received data


In [42]:
df.shape

(11191, 7)

In [37]:
df_merge = df[df['year'] == 2022].merge(df[df['year'] == 2023], on=['be_bid','ap_data_sector_id','ap_data_sector_name'],how='inner')

In [48]:
df[(df['year'] == 2022)].groupby(by=['stage_lid']).count()

,ap_data_sector_id,ap_data_sector_name,year,be_bid,fp_be_bid,mp_be_bid
stage_lid,,,,,,


In [39]:
df[(df['year'] == 2023)].groupby(by=['stage_lid']).count()

,ap_data_sector_id,ap_data_sector_name,year,be_bid,fp_be_bid,mp_be_bid
stage_lid,,,,,,


In [23]:
df_adv = df_merge[df_merge['stage_lid_x'] < df_merge['stage_lid_y']]
df_adv[df_adv['stage_lid_x'] == 4]

,ap_data_sector_id,ap_data_sector_name,year_x,be_bid,fp_be_bid_x,mp_be_bid_x,stage_lid_x,year_y,fp_be_bid_y,mp_be_bid_y,stage_lid_y
26,9,SOY_BRAZIL_SUMMER,2022,ESY2200565223,ESY2200283549,None,4,2023,ESY2200283549,None,5
27,9,SOY_BRAZIL_SUMMER,2022,ESY2200565215,ESY2200267798,None,4,2023,ESY2200267798,None,5
28,9,SOY_BRAZIL_SUMMER,2022,ESY2200565205,ESY2200267732,None,4,2023,ESY2200267732,None,5
29,9,SOY_BRAZIL_SUMMER,2022,ESY2200565197,ESY2200267806,None,4,2023,ESY2200267806,None,5
30,9,SOY_BRAZIL_SUMMER,2022,ESY2200565165,ESY2200283649,None,4,2023,ESY2200283649,None,5
31,9,SOY_BRAZIL_SUMMER,2022,ESY2200565007,ESY2200283817,None,4,2023,ESY2200283817,None,5
32,9,SOY_BRAZIL_SUMMER,2022,ESY2200564899,ESY2200264651,None,4,2023,ESY2200264651,None,5
33,9,SOY_BRAZIL_SUMMER,2022,ESY2200564865,ESY2200267771,None,4,2023,ESY2200267771,None,5
34,9,SOY_BRAZIL_SUMMER,2022,ESY2200564800,ESY2200283558,None,4,2023,ESY2200283558,None,5
35,9,SOY_BRAZIL_SUMMER,2022,ESY2200564770,ESY2200264638,None,4,2023,ESY2200264638,None,5


In [ ]:
df_cpifl_table = pd.read_parquet('s3://us.com.syngenta.ap.nonprod/uat/dme/performance/compute_trial_checks/data/SOY_NA_SUMMER/SingleExp/2021/trial_checks.parquet')
cpifl_group_cols = [
    'ap_data_sector', 'analysis_type', 'analysis_year',
    'source_id','entry_id', 'material_type'
]

cpifl_info_cols = ['cpifl', 'cperf', 'cagrf', 'cmatf', 'cregf', 'crtnf']
cpifl_keep_cols = cpifl_group_cols.copy()
cpifl_keep_cols.extend(cpifl_info_cols)
df_cpifl_grouped = df_cpifl_table[cpifl_keep_cols].groupby(by=cpifl_group_cols).max().reset_index()

In [ ]:
# run code to get trial_pheno_data
df_trial_pheno = \
    performance_sql_recipes.get_trial_pheno_data_reduced_columns(ap_data_sector, int(analysis_year))

# remove columns like et_value, plot_barcode, etc.
# this could be done in the sql query, which would be faster and less memory intensive.
df_trial_pheno = df_trial_pheno[[
    'ap_data_sector',
    'analysis_year',
    'trial_id',
    'entry_id',
    'source_id',
    'market_segment',
    'trait',
    'maturity_group',
    'dme_chkfl',
    'irrigation',
    'et_value',
    'result_numeric_value'
]]

# merge in check flags per trait
df_trial_pheno = df_trial_pheno.merge(
    df_cpifl_grouped,
    on=['ap_data_sector', 'analysis_year', 'source_id', 'entry_id'],
    how='left'
).rename(columns={'market_segment': 'market_seg'})

df_trial_pheno = performance_helper.get_chkfl(df_trial_pheno)

df_trial_pheno[['cpifl', 'chkfl']] = df_trial_pheno[['cpifl', 'chkfl']].fillna(value=0)
df_trial_pheno['material_type'] = df_trial_pheno['material_type'].fillna(value='entry')

# get trait values relative to check and trial mean
# if check mean does not exist, use trial mean
trial_group_cols = ['ap_data_sector','analysis_year','trial_id','trait']
score_col = 'result_numeric_value'
keep_cols = trial_group_cols.copy()
keep_cols.append(score_col)

# check mean for each trait
df_trial_pheno_checks_mean = df_trial_pheno[df_trial_pheno['chkfl']==1][keep_cols].groupby(
    by=trial_group_cols
).mean().reset_index() # chkfl is per trait
df_trial_pheno_checks_mean = df_trial_pheno_checks_mean.rename(
    columns={'result_numeric_value': 'trial_mean_value'}
)

In [ ]:
df = df_trial_pheno[df_trial_pheno['trait'] == 'MRTYN']
df['result_numeric_value'][df['result_numeric_value'] < 0] = np.nan
print(df.shape)

In [ ]:
df_checks = df[df['chkfl'] == 1]

In [ ]:
plt.plot(df['maturity_group'],df['result_numeric_value'],'.')
plt.show()

In [ ]:
df_checks.head()

In [ ]:
df_checks[df_checks['trial_id'] == '21SUPSLYG6393340']

In [ ]:
df

In [ ]:
DKU_DST_ap_data_sector = 'CORN_BRAZIL_SUMMER'
DKU_DST_analysis_year = 2019
DKU_DST_analysis_type = 'GenoPred'

In [ ]:
with DenodoConnection() as dc:
    data_sector_config = dc.get_data("""
                        SELECT 
                            ap_data_sector_name,
                            spirit_crop_guid,
                            entry_id_source
                          FROM "managed"."rv_ap_data_sector_config"
                        WHERE "ap_data_sector_name" = {0}""".format("'" + DKU_DST_ap_data_sector + "'"))

checks_df = merge_trial_check_entries(DKU_DST_ap_data_sector,
                                      DKU_DST_analysis_year,
                                      'GenoPred',
                                      data_sector_config["spirit_crop_guid"].iloc[0],
                                      data_sector_config["entry_id_source"].iloc[0])

In [ ]:
checks_df.shape

In [ ]:
df = get_data()

In [ ]:
df.shape

In [ ]:
value_cols = ['result_numeric_value']
meta_cols = ['year','be_bid','trial_id','plot_barcode']
var_cols = ['trait_measure_code']
df_piv = df.pivot_table(
    values=value_cols,
    index=meta_cols,
    columns=var_cols
).reset_index()

# rename columns and reset index
df_piv_cols = []
for col_tuple in df_piv.columns:
    if col_tuple[0] == 'value' or col_tuple[0] == 'alpha_value' or col_tuple[0] == 'result_numeric_value':
        df_piv_cols.append('_'.join(filter(None, col_tuple[1:])).strip('_'))
    else:
        df_piv_cols.append('_'.join(filter(None, col_tuple)).strip('_'))
        
df_piv.columns = df_piv_cols

In [ ]:
df_piv.count()